In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

c:\Users\s1612415\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(1)

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

train_dataloader2 = DataLoader(training_data, batch_size=batch_size)
test_dataloader2 = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
class LowRankLinearLayer(nn.Module):
    def __init__(self, initial_weight, initial_bias, rank):
        super(LowRankLinearLayer, self).__init__()
        U, S, Vh = torch.linalg.svd(initial_weight, full_matrices=False)
        self.U = nn.Parameter(U[:, :rank])
        self.S = nn.Parameter(S[:rank])
        self.Vh = nn.Parameter(Vh[:rank, :])
        self.bias = nn.Parameter(initial_bias)

    def forward(self, x):
        return torch.einsum('mn, bn -> bm', self.U @ torch.diag(self.S) @ self.Vh, x) + self.bias


    
class LowRankNeuralNetwork(nn.Module):
    def __init__(self, prec):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            LowRankLinearLayer(model._modules['linear_relu_stack'][0].weight, model._modules['linear_relu_stack'][0].bias, int(prec * 512)),
            nn.ReLU(),
            LowRankLinearLayer(model._modules['linear_relu_stack'][2].weight, model._modules['linear_relu_stack'][2].bias, int(prec * 512)),
            nn.ReLU(),
            LowRankLinearLayer(model._modules['linear_relu_stack'][4].weight, model._modules['linear_relu_stack'][4].bias, int(prec * 10))
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model2 = LowRankNeuralNetwork(0.6).to(device)
print(model2)

LowRankNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): LowRankLinearLayer()
    (1): ReLU()
    (2): LowRankLinearLayer()
    (3): ReLU()
    (4): LowRankLinearLayer()
  )
)


In [7]:
def count_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        print(f'{name}: {params}')
        total_params+=params
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [8]:
count_parameters(model)

linear_relu_stack.0.weight: 401408
linear_relu_stack.0.bias: 512
linear_relu_stack.2.weight: 262144
linear_relu_stack.2.bias: 512
linear_relu_stack.4.weight: 5120
linear_relu_stack.4.bias: 10
Total Trainable Params: 669706


669706

In [9]:
count_parameters(model2)

linear_relu_stack.0.U: 157184
linear_relu_stack.0.S: 307
linear_relu_stack.0.Vh: 240688
linear_relu_stack.0.bias: 512
linear_relu_stack.2.U: 157184
linear_relu_stack.2.S: 307
linear_relu_stack.2.Vh: 157184
linear_relu_stack.2.bias: 512
linear_relu_stack.4.U: 60
linear_relu_stack.4.S: 6
linear_relu_stack.4.Vh: 3072
linear_relu_stack.4.bias: 10
Total Trainable Params: 717026


717026

In [10]:
loss_fn = nn.CrossEntropyLoss()
loss_fn2 = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer2 = torch.optim.SGD(model2.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    
    train(train_dataloader2, model2, loss_fn2, optimizer2)
    test(test_dataloader2, model2, loss_fn2)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294026  [    0/60000]
loss: 2.285374  [ 6400/60000]
loss: 2.261674  [12800/60000]
loss: 2.258286  [19200/60000]
loss: 2.246893  [25600/60000]
loss: 2.204551  [32000/60000]
loss: 2.224775  [38400/60000]
loss: 2.186471  [44800/60000]
loss: 2.188018  [51200/60000]
loss: 2.143335  [57600/60000]
Test Error: 
 Accuracy: 34.9%, Avg loss: 2.144163 

loss: 2.299525  [    0/60000]
loss: 2.295399  [ 6400/60000]
loss: 2.289398  [12800/60000]
loss: 2.287983  [19200/60000]
loss: 2.290339  [25600/60000]
loss: 2.271779  [32000/60000]
loss: 2.283621  [38400/60000]
loss: 2.272868  [44800/60000]
loss: 2.274235  [51200/60000]
loss: 2.261751  [57600/60000]
Test Error: 
 Accuracy: 24.1%, Avg loss: 2.262018 

Epoch 2
-------------------------------
loss: 2.155175  [    0/60000]
loss: 2.143347  [ 6400/60000]
loss: 2.081628  [12800/60000]
loss: 2.104539  [19200/60000]
loss: 2.053961  [25600/60000]
loss: 1.977515  [32000/60000]
loss: 2.022094  [38400/60000]
loss: 

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

torch.save(model2.state_dict(), "model2.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
Saved PyTorch Model State to model.pth
